<a href="https://colab.research.google.com/github/OmraOneil/100-days-of-code/blob/master/Copie_de_perasVsManzanas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparando peras con manzanas: una introducción al Deep Learning

<table><tr>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pera.png" width=500px></td>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/manzana.png" width=500px></td>
</tr></table>

En este notebook vamos a experimentar un poco con las redes neuronales profundas (Deep Learning) para resolver una tarea divertida: ¡vamos a construir una IA que distinga fotos de **peras** y de **manzanas**!

Si nunca has usado un notebook de Google Colaboratory verás que es muy sencillo: avanza leyendo el texto y cuando encuentres una sección con código puedes ejecutarla haciendo click en el icono del triángulo que aparece cuando colocas el cursos del ratón sobre el código. También puedes desplazarte por el notebook con las flechas direccionales y ejecutar una sección de código pulsando Mayúsculas + Enter.

## Preparando el entorno

Para poder entrenar una red neuronal en un tiempo razonable es fundamental contar con una GPU (Graphics Processing Unit). Este tipo de hardware está especialmente diseñado para realizar operaciones de red neuronal de forma muy eficiente, ahorrando así tiempos de cálculo y costes. Conectar una GPU a la instancia de Google en la que corre este notebook es muy sencillo:

1. En la barra de menús arriba a la izquierda, haz click en la opción "Entorno de ejecución".
2. En el desplegable, selecciona "Cambiar tipo de entorno de ejecución".
3. En la venta que aparece, en el desplegable "Acelerador por hardware" selecciona "GPU".

¡Listo! Podemos comprobar la GPU que se nos ha asignado ejecutando el siguiente comando.

In [ ]:
!nvidia-smi

## Adquisición de datos

Vamos a utilizar un conjunto de datos de fotos de frutas disponible en [Kaggle](https://www.kaggle.com/moltean/fruits). Para poder descargar estos datos normalmente sería necesario crearse una cuenta en Kaggle y obtener unas credenciales; además necesitaríamos procesar estos datos para quedarnos solo con las fotos de peras y manzanas de entre todas las fotos de frutas disponibles. Para facilitar este ejercicio ya disponemos de una versión lista para usar de estos datos, que podemos descargar usando el siguiente comando.

In [ ]:
!wget https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/perasVSmanzanas.tgz

Ahora vamos a descomprimir los datos que hemos descargado, usando el siguiente comando

In [ ]:
!tar -xvzf perasVSmanzanas.tgz

Podemos comprobar que los datos se han descargado completamente seleccionando el icono de la carpeta que aparece a la izquierda, lo cual nos mostrará en el lateral un explorador de ficheros. Deberían poder verse una carpeta `train_data` con los datos que usaremos para que la red neuronal aprenda, y otra carpeta `test_data` con los datos que emplearemos para comprobar si la red funciona. Si no ves estas carpetas, haz click en el icono de `Actualizar`.

Vamos a visualizar las imágenes de peras y manzanas que tenemos. Para ello necesitamos instalar el paquete ipyplot de python.

In [ ]:
!pip install ipyplot

El siguiente código nos mostrará una pequeña interfaz donde visualizar una muestra de las imágenes que trabajaremos. Puedes cambiar entre peras y manzanas usando las pestañas que se muestran arriba a la izquierda.

In [ ]:
from glob import glob
import ipyplot
import numpy as np

all_images = glob("./train_data/*/*.jpg")  # Get all image paths
np.random.shuffle(all_images)  # Randomize to show different images each run
all_labels = [f.split("/")[-2] for f in all_images]  # Extract class names from path

ipyplot.plot_class_tabs(all_images, all_labels, max_imgs_per_tab=60, img_width=100, force_b64=True)

Estas fotos se han obtenido eliminando el fondo de la imagen con un proceso automático, por lo que puede que en en algunos casos se aprecien bordes algo mal definidos. Igualmente, podemos trabajar con este conjunto de imágenes.

## Cargando las imágenes

Para que la red neuronal pueda aprender de las imágenes, primero necesitamos construir un `Dataset` en base a ellas. Un `Dataset` es un objeto que se encarga de ir cargando las imágenes de disco cuando las necesitamos, evitando así tenerlas todas en memoria al mismo tiempo. Además nos facilita algunas tareas de procesado de las imágenes.

Podemos crear un  `Dataset` con todas las imágenes de la carpeta de entrenamiento de la siguiente manera

In [ ]:
from keras.preprocessing import image_dataset_from_directory

image_size = 100
batch_size = 64

train_dataset = image_dataset_from_directory(
    "./train_data", 
    image_size = (image_size, image_size),
    batch_size = batch_size, 
    label_mode = 'binary'
)

Los parámetros con los que se configura el dataset son los siguientes:

* El **directorio** donde encontrar las imágenes.
* El **tamaño de imagen** al que se redimensionarán las imágenes al cargarlas en memoria, en este caso 100x100 píxeles.
* EL **tamaño de los batches** de imágenes que se generarán. La red neuronal irá procesando imágenes en grupos de este tamaño, evitando así mantenerlas todas en memoria al mismo tiempo.
* El **modo de etiquetado**, esto es, qué clase de problema de clasificación estamos resolviendo en este caso. Aquí queremos tomar una decisión binaria (peras VS manzanas), por lo que elegimos el modo binario.

Un `Dataset` funciona como un generador de python, lo que significa que podemos iterar sobre él para ir obteniendo cada uno de los batches o grupos de imágenes. Por ejemplo, con el código siguiente podemos observar el primer batch.

In [ ]:
for X_batch, y_batch in train_dataset:
    print(f"Shape of input batch: {X_batch.shape}")
    print(f"Shape of output batch: {y_batch.shape}")
    print(f"Input batch:\n{X_batch}")
    print(f"Output batch:\n{y_batch}")
    break

Podemos ver cómo el generador produce un tensor de entrada conteniendo los datos de un batch de imágenes, así como tensor de salida en el que manzanas y peras se han codificado con las etiquetas 0 y 1. Las imágenes se representan como una serie de píxeles en el rango `[0, 255]`, representando el valor de intensidad de cada píxel.

Vamos ahora a crear otro `Dataset` con los datos de test

In [ ]:
test_dataset = image_dataset_from_directory(
    "./test_data", 
    batch_size = batch_size, 
    image_size = (image_size, image_size),
    label_mode = 'binary'
)

## Construyendo nuestra primera red neuronal

Ahora que tenemos nuestros datos, ¡vamos a entrenar una red neuronal! El siguiente código diseña la red usando la librería de Deep Learning [Keras](https://keras.io/)

In [ ]:
# Importamos los elementos de Keras que necesitamos
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.experimental.preprocessing import Rescaling

# Inicializamos la red neuronal como de tipo Secuencial: cada elemento que añadamos a la red se conecta al declarado anteriormente
model = Sequential()
# Añadimos una capa que normaliza los valores de los píxeles, de [0, 255] a [0, 1]
model.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
# Añadimos una capa convolucional para que la red aprenda a detectar bordes
model.add(Convolution2D(32, 3, activation='relu'))
# Añadimos una capa de pooling para focalizar el detector anterior
model.add(MaxPooling2D(2))
# Aplanamos las imágenes resultado
model.add(Flatten())
# Añadimos una unidad final de salida, que nos dará un valor entre 0 y 1 para indicar si la imagen es pera o manzana
model.add(Dense(1, activation='sigmoid'))

# Compilamos la red neuronal: en este paso Keras analiza nuestro diseño y genera código CUDA para que la red pueda entrenarse
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=["accuracy"])
# Mostramos un resumen de la red construída
model.summary()

Una vez diseñada la red, podemos hacer que aprenda usando el `Dataset` de entrenamiento. Vamos a realizar 1 época, esto es, la red neuronal aprenderá observando las imágenes de entrenamiento una sola vez.

In [ ]:
model.fit(train_dataset, epochs=1)

Una vez entrenada, podemos medir el acierto de la red sobre los datos de test.

In [ ]:
loss, acc = model.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

¡Conseguimos más de un 80% de acierto! No está mal, ¡pero podemos hacerlo mejor!

## Creando una red más profunda

Vamos a crear una red más profunda, usando más capas de Convolución y MaxPooling, de manera que así la red neuronal pueda identificar elementos más complejos en la imagen. Asímismo, vamos a añadir una capa Densa tras el aplanado para que las características visuales detectadas puedan procesarse de forma más avanzada. También vamos a añadir una capa de Dropout para robustecer la red neuronal y que sea más precisa. Finalmente, enseñaremos cada imagen 10 veces a la red neuronal para mejorar su aprendizaje.

In [ ]:
# Importamos algunos objetos más de Keras
from keras.layers.core import Dropout

# Initializamos la red
model = Sequential()
# Añadimos una capa que normaliza los valores de los píxeles, de [0, 255] a [0, 1]
model.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
# Añadimos una capa convolucional para que la red aprenda a detectar bordes
model.add(Convolution2D(32, 3, activation='relu'))
# Añadimos una capa de pooling para focalizar el detector anterior
model.add(MaxPooling2D(2))
# Añadimos una segunda capa convolucional para que la red aprenda a detectar elementos más complejos, como figuras geométricas
model.add(Convolution2D(64, 3, activation='relu'))
# Añadimos una capa de pooling para focalizar el detector anterior
model.add(MaxPooling2D(2))
# Añadimos una tercera capa convolucional para que la red aprenda a detectar elementos aún más complejos, como patrones formados por figuras geométricas
model.add(Convolution2D(128, 3, activation='relu'))
# Añadimos una capa de pooling para focalizar el detector anterior
model.add(MaxPooling2D(2))
# Aplanamos las imágenes resultado
model.add(Flatten())
# Añadimos una capa densa para procesar las características visuales detectadas por la capa anterior
model.add(Dense(64, activation='relu'))
# Añadimos Dropout al 50% para que la red se acostumbre a imágenes que puedan contener fallos
model.add(Dropout(0.5))
# Añadimos una unidad final de salida, que nos dará un valor entre 0 y 1 para indicar si la imagen es pera o manzana
model.add(Dense(1, activation='sigmoid'))

# Compilamos la red neuronal
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
# Mostramos un resumen de la red construída
model.summary()

# Hacemos que la red aprenda
model.fit(train_dataset, epochs=10)

# Calculamos el acierto sobre los datos de test
loss, acc = model.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

¡Una red más profunda produce resultados mucho mejores!

## Transfer learning

Podemos mejorar aún más la red anterior si en lugar de diseñarla nosotros desde 0 partimos de un diseño de red ya existente. La librería Keras incluye varios de estos diseños en su módulo [Keras Applications](https://keras.io/api/applications/); en esta ocasión vamos a usar la red VGG16.

In [ ]:
from keras.applications import VGG16

vgg16_model = VGG16(include_top=False, input_shape=(image_size, image_size, 3))

Podemos ver cómo está construída esta red. ¡Es mucho más profunda que las que hemos hecho antes!

In [ ]:
vgg16_model.summary()

Por defecto todas las redes de Keras Applications vienen precargadas con los parámetros con los que fueron entrenadas: [el dataset ImageNet](http://www.image-net.org/). Este dataset está conformado por multitud de imágenes naturales, lo que ayuda a que la red ya sea capaz de reconocer bastantes elementos gráficos y objetos. Para preservar ese conocimiento en la red neuronal, vamos a marcar que durante el proceso de aprendizaje que haremos a continuación estos parámetros no puedan variar.

In [ ]:
vgg16_model.trainable = False

Ahora vamos a construir nuestra red neuronal propia, usando la VGG16 pre-entrenada como parte de ella. Empezamos construyendo la red con una capa que normalice las imágenes al formato esperado por la VGG16, después añadimos la propia VGG16, y continuación aplanamos y repetimos el patrón de las redes que hemos diseñado antes.

In [ ]:
# Importamos algunos objetos más de Keras
from keras.applications.vgg16 import preprocess_input
from keras.layers import Lambda

# Initializamos la red
model = Sequential()
# Añadimos una capa que realice la normalización de imágenes que requiere VGG16
model.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))
# Añadimos la VGG16
model.add(vgg16_model)
# Aplanamos las imágenes resultado
model.add(Flatten())
# Añadimos una capa densa para procesar las características visuales detectadas por la capa anterior
model.add(Dense(64, activation='relu'))
# Añadimos Dropout al 50% para que la red se acostumbre a imágenes que puedan contener fallos
model.add(Dropout(0.5))
# Añadimos una unidad final de salida, que nos dará un valor entre 0 y 1 para indicar si la imagen es pera o manzana
model.add(Dense(1, activation='sigmoid'))

# Compilamos la red neuronal
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
# Mostramos un resumen de la red construída
model.summary()

# Hacemos que la red aprenda
model.fit(train_dataset, epochs=10)

# Calculamos el acierto sobre los datos de test
loss, acc = model.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

¡Reutilizando una red pre-entrenada hemos conseguido un nivel de acierto casi perfecto!

## Probando con otras imágenes

¿No te fías de que la red neuronal que hemos hecho funcione? ¡Pruébalo con la foto que quieras! A continuación definimos la función `download_image` que dada la URL de una imagen la guarda como una variable en python.

In [ ]:
from io import BytesIO
import numpy as np
import requests

from PIL import Image

def download_image(image_url):
    return Image.open(requests.get(image_url, stream=True).raw)

Aquí tienes algunas URL de imágenes que puedes usar:

Manzanas:

* https://upload.wikimedia.org/wikipedia/commons/2/25/Alice_%28apple%29.jpg
* https://upload.wikimedia.org/wikipedia/commons/d/d2/Malus-Boskoop_organic.jpg

Peras

* https://upload.wikimedia.org/wikipedia/commons/2/2d/PearPhoto.jpg
* https://upload.wikimedia.org/wikipedia/commons/thumb/5/5d/Pear_DS.jpg/1920px-Pear_DS.jpg
* https://upload.wikimedia.org/wikipedia/commons/c/cf/Pears.jpg

Cambia la URL que aparece a continuación y ejecuta el código para descargar la imagen y mostrarla.

In [ ]:
img = download_image("https://upload.wikimedia.org/wikipedia/commons/2/25/Alice_%28apple%29.jpg")
img

Dado que la imagen que has elegido puede venir en cualquier formato y tamaño, vamos a definir la siguiente función para que normalice esa imagen y la convierta al formato adecuado para la red neuronal

In [ ]:
import numpy as np
from skimage.transform import resize

def preprocess_image(img):
    img = np.array(img.resize((100, 100)))
    return np.expand_dims(img, axis=0)

Con el siguiente código aplicamos la normalización sobre la imagen que hemos descargado, obteniendo así `img_ready`

In [ ]:
img_ready = preprocess_image(img)
img_ready

Ahora analizamos la imagen con la red neuronal para que nos dé una respuesta

In [ ]:
probability = model.predict(img_ready)
probability

¿Qué significa el valor `probability` que hemos obtenido? Es la confianza que tiene la red neuronal en que la imagen represente una pera o una manzana. Si el valor devuelto es cercano a `0`, la red está muy segura de que la imagen muestra una manzana, en cambio si el valor es cercano a `1` está convencida de que se trata de una pera. El siguiente código decide cortar esta probabilidad en el punto medio (`0.5`) y en base a eso enunciar si estamos ante una manzana o pera.

In [ ]:
label = 1 if probability[0][0] > 0.5 else 0
train_dataset.class_names[label]

## Cierre

Y... ¿esto es todo amigos? ¡No! El desafío que nos hemos propuesto es sencillo, pero aún así la red neuronal profunda que hemos construído es aún mejorable. Si le pasamos una fotografía de una fruta con mala iluminación o con otros elementos presentes que sean distractores, probablemente la red neuronal falle. Existen muchas técnicas que podemos usar para robustecer nuestra red ante estos problemas, y mucho más que aprender en el mundo de las redes neuronales profundas. ¡Esto es solo el comienzo! :)